# Study of a Facebook messenger conversation

In [ ]:
from bs4 import BeautifulSoup
import urllib

yourName = 'Thomas Marchand'
url = r"YOURPATH/13.html"
page = open(url, encoding="utf8")
soup = BeautifulSoup(page.read(), 'lxml')


### Getting the conversation name & participants

In [ ]:
def getConvName(soup):
    nameConv = soup.h3.get_text()
    return(nameConv)

In [ ]:
def getListParticipants(soup):
    
    def getStringParticipants(soup):
        stringClassThread = str(soup\
                            .find_all("div", class_="thread")
                           )
        intermediate = stringClassThread.split('</h3>')
        return(intermediate[1]\
               .split('<div class="message">')[0]
              )
    
    stringClassThread = str(soup\
                        .find_all("div", class_="thread")
                       )
    stringParticipants = getStringParticipants(soup)
    listParticipants = stringParticipants.split(",")
    listParticipants[0] = listParticipants[0].split(":")[1]
    listParticipants = [Participant[1:] 
                    for Participant in listParticipants]
    listParticipants.append(yourName)
    return(listParticipants)

### Getting the number of messages

In [ ]:
def getNumberOfMessage(soup):
    return(len(soup.find_all("div", class_="message")))

In [ ]:
def getWriter(soup, messageIndex):
    return(soup\
           .find_all("div", class_="message")[messageIndex]\
           .find_all("span", class_="user")[0].get_text())

In [ ]:
def getNumberOfMessgePerUser(soup):
    userList = getListParticipants(soup)
    userActivityDict = {u:0 for u in userList}
    allMessages = (soup\
           .find_all("div", class_="message"))
    for i in range(getNumberOfMessage(soup)):
        tempWriter = allMessages[i]\
                .find_all("span", class_="user")[0]\
                .get_text()
        if tempWriter not in userActivityDict:
            userActivityDict[tempWriter] = 0
        userActivityDict[tempWriter] += 1
    return(userActivityDict)